In [68]:
import logging

import gensim
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
from tqdm import tqdm

tqdm.pandas()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Load Data

In [2]:
# global variable
CATEGORY = "Grocery_and_Gourmet_Food"
DATA_PATH = "data/evaluation"

In [3]:
train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [4]:
train.head()

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...


# Preparing Review Text for LDA

In [5]:
processed_reviews = train["processedReviewText"].progress_apply(lambda x: x.split())

100%|█████████████████████████████████████████████████████| 47774/47774 [00:00<00:00, 138101.14it/s]


In [6]:
processed_reviews

0        [curry, paste, delicious, curry, fry, chicken,...
1        [purchase, different, curry, grocery, store, c...
2        [start, new, diet, restrict, added, sugar, bra...
3        [flavor, begin, tell, love, mae, ploy, curry, ...
4        [use, lot, recently, chicken, dish, use, lot, ...
                               ...                        
47769    [quality, reese, peanut, butter, cup, product,...
47770    [purchase, husband, love, reeses, valentine, d...
47771    [stuff, incredibily, yummy, good, regular, bro...
47772    [nice, aroma, body, taste, buy, coffee, good, ...
47773    [diabetic, accord, wife, doctor, order, intent...
Name: processedReviewText, Length: 47774, dtype: object

In [7]:
# creating a bag-of-words
dictionary = gensim.corpora.Dictionary(processed_reviews)

2021-08-23 15:40:28,605 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-08-23 15:40:28,961 : INFO : adding document #10000 to Dictionary(14898 unique tokens: ['add', 'chicken', 'coconut', 'curry', 'delicious']...)
2021-08-23 15:40:29,318 : INFO : adding document #20000 to Dictionary(21785 unique tokens: ['add', 'chicken', 'coconut', 'curry', 'delicious']...)
2021-08-23 15:40:29,703 : INFO : adding document #30000 to Dictionary(27143 unique tokens: ['add', 'chicken', 'coconut', 'curry', 'delicious']...)
2021-08-23 15:40:30,120 : INFO : adding document #40000 to Dictionary(31844 unique tokens: ['add', 'chicken', 'coconut', 'curry', 'delicious']...)
2021-08-23 15:40:30,477 : INFO : built Dictionary(34875 unique tokens: ['add', 'chicken', 'coconut', 'curry', 'delicious']...) from 47774 documents (total 1942617 corpus positions)
2021-08-23 15:40:30,494 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(34875 unique tokens: ['add', 'chicken', 'coconut', 'curry'

In [8]:
# filtering out tokens that appear in less than 50 reviews
# or more than 0.75 of the corpus
dictionary.filter_extremes(no_below=15, no_above=0.75)

2021-08-23 15:40:30,572 : INFO : discarding 29213 tokens: [('lidded', 9), ('scrambled', 4), ('yogurts', 1), ('chaokoh', 5), ('icer', 2), ('newbie', 11), ('whomever', 6), ('curris', 1), ('eggnog', 13), ('exclude', 12)]...
2021-08-23 15:40:30,573 : INFO : keeping 5662 tokens which were in no less than 15 and no more than 35830 (=75.0%) documents
2021-08-23 15:40:30,590 : INFO : resulting dictionary: Dictionary(5662 unique tokens: ['add', 'chicken', 'coconut', 'curry', 'delicious']...)


In [9]:
# creating dict how many words and time the word appear
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_reviews]

In [10]:
# checking first review
processed_reviews[0]

['curry',
 'paste',
 'delicious',
 'curry',
 'fry',
 'chicken',
 'vegetable',
 'add',
 'coconut',
 'milk',
 'delicious',
 'dish',
 'spicy',
 'look',
 'mild',
 'look']

In [11]:
bow_doc_first = bow_corpus[0]

for i in range(len(bow_doc_first)):
    print(f"Word {bow_doc_first[i][0]} ('{dictionary[i]}') appears {bow_doc_first[i][1]} times.")

Word 0 ('add') appears 1 times.
Word 1 ('chicken') appears 1 times.
Word 2 ('coconut') appears 1 times.
Word 3 ('curry') appears 2 times.
Word 4 ('delicious') appears 2 times.
Word 5 ('dish') appears 1 times.
Word 6 ('fry') appears 1 times.
Word 7 ('look') appears 2 times.
Word 8 ('mild') appears 1 times.
Word 9 ('milk') appears 1 times.
Word 10 ('paste') appears 1 times.
Word 11 ('spicy') appears 1 times.
Word 12 ('vegetable') appears 1 times.


In [12]:
# creating tf-idf model object

tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

2021-08-23 15:40:31,822 : INFO : collecting document frequencies
2021-08-23 15:40:31,823 : INFO : PROGRESS: processing document #0
2021-08-23 15:40:31,916 : INFO : PROGRESS: processing document #10000
2021-08-23 15:40:31,969 : INFO : PROGRESS: processing document #20000
2021-08-23 15:40:32,028 : INFO : PROGRESS: processing document #30000
2021-08-23 15:40:32,088 : INFO : PROGRESS: processing document #40000
2021-08-23 15:40:32,135 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 47774 documents and 5662 features (1445442 matrix non-zeros)', 'datetime': '2021-08-23T15:40:32.135509', 'gensim': '4.0.1', 'python': '3.9.6 (default, Jun 29 2021, 05:25:02) \n[Clang 12.0.5 (clang-1205.0.22.9)]', 'platform': 'macOS-11.4-x86_64-i386-64bit', 'event': 'initialize'}


# Training LDA Model

In [13]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=50, id2word=dictionary, passes=20, workers=8)

2021-08-23 15:40:32,140 : INFO : using symmetric alpha at 0.02
2021-08-23 15:40:32,141 : INFO : using symmetric eta at 0.02
2021-08-23 15:40:32,143 : INFO : using serial LDA version on this node
2021-08-23 15:40:32,167 : INFO : running online LDA training, 50 topics, 20 passes over the supplied corpus of 47774 documents, updating every 16000 documents, evaluating every ~47774 documents, iterating 50x with a convergence threshold of 0.001000
2021-08-23 15:40:32,169 : INFO : training LDA model using 8 processes
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/proje

2021-08-23 15:40:45,419 : INFO : topic #10 (0.020): 0.004*"good" + 0.004*"like" + 0.004*"coffee" + 0.004*"great" + 0.004*"product" + 0.004*"buy" + 0.004*"love" + 0.004*"flavor" + 0.004*"chocolate" + 0.004*"almond"
2021-08-23 15:40:45,421 : INFO : topic #25 (0.020): 0.005*"coffee" + 0.005*"chocolate" + 0.004*"flavor" + 0.004*"cup" + 0.004*"use" + 0.004*"like" + 0.004*"tea" + 0.004*"product" + 0.004*"taste" + 0.004*"great"
2021-08-23 15:40:45,422 : INFO : topic diff=1.273746, rho=0.316228
2021-08-23 15:40:45,795 : INFO : PROGRESS: pass 0, dispatched chunk #19 = documents up to #40000/47774, outstanding queue size 3
2021-08-23 15:40:46,327 : INFO : PROGRESS: pass 0, dispatched chunk #20 = documents up to #42000/47774, outstanding queue size 3
2021-08-23 15:40:46,921 : INFO : PROGRESS: pass 0, dispatched chunk #21 = documents up to #44000/47774, outstanding queue size 3
2021-08-23 15:40:47,483 : INFO : PROGRESS: pass 0, dispatched chunk #22 = documents up to #46000/47774, outstanding queue

2021-08-23 15:41:01,550 : INFO : topic #37 (0.020): 0.017*"cracker" + 0.009*"cheese" + 0.008*"ritz" + 0.006*"taste" + 0.006*"coffee" + 0.005*"good" + 0.005*"cheddar" + 0.005*"like" + 0.005*"snack" + 0.004*"buy"
2021-08-23 15:41:01,551 : INFO : topic #0 (0.020): 0.013*"bar" + 0.008*"chocolate" + 0.007*"peanut" + 0.007*"snack" + 0.006*"cereal" + 0.006*"granola" + 0.006*"like" + 0.006*"eat" + 0.005*"taste" + 0.005*"good"
2021-08-23 15:41:01,551 : INFO : topic #10 (0.020): 0.005*"almond" + 0.005*"chocolate" + 0.005*"like" + 0.005*"good" + 0.005*"love" + 0.005*"taste" + 0.004*"keebler" + 0.004*"product" + 0.004*"buy" + 0.004*"great"
2021-08-23 15:41:01,552 : INFO : topic #45 (0.020): 0.009*"ring" + 0.007*"cashew" + 0.007*"chocolate" + 0.006*"square" + 0.005*"taste" + 0.005*"product" + 0.004*"sugar" + 0.004*"good" + 0.004*"like" + 0.004*"great"
2021-08-23 15:41:01,553 : INFO : topic diff=0.276872, rho=0.196544
2021-08-23 15:41:02,424 : INFO : -11.327 per-word bound, 2568.8 perplexity estimat

2021-08-23 15:41:15,735 : INFO : PROGRESS: pass 3, dispatched chunk #2 = documents up to #6000/47774, outstanding queue size 3
2021-08-23 15:41:16,190 : INFO : PROGRESS: pass 3, dispatched chunk #3 = documents up to #8000/47774, outstanding queue size 3
2021-08-23 15:41:16,649 : INFO : PROGRESS: pass 3, dispatched chunk #4 = documents up to #10000/47774, outstanding queue size 3
2021-08-23 15:41:17,073 : INFO : PROGRESS: pass 3, dispatched chunk #5 = documents up to #12000/47774, outstanding queue size 3
2021-08-23 15:41:17,530 : INFO : PROGRESS: pass 3, dispatched chunk #6 = documents up to #14000/47774, outstanding queue size 3
2021-08-23 15:41:18,123 : INFO : PROGRESS: pass 3, dispatched chunk #7 = documents up to #16000/47774, outstanding queue size 3
2021-08-23 15:41:18,701 : INFO : PROGRESS: pass 3, dispatched chunk #8 = documents up to #18000/47774, outstanding queue size 2
2021-08-23 15:41:19,158 : INFO : PROGRESS: pass 3, dispatched chunk #9 = documents up to #20000/47774, out

2021-08-23 15:41:32,474 : INFO : topic #19 (0.020): 0.019*"mojito" + 0.009*"crystal" + 0.008*"peppercorn" + 0.008*"melitta" + 0.008*"sprout" + 0.007*"goji" + 0.007*"pepper" + 0.007*"disappointing" + 0.007*"grandkids" + 0.006*"buy"
2021-08-23 15:41:32,475 : INFO : topic #11 (0.020): 0.018*"reorder" + 0.007*"kcups" + 0.007*"soup" + 0.007*"comfort" + 0.007*"soooo" + 0.007*"emergency" + 0.007*"china" + 0.006*"locally" + 0.006*"shopping" + 0.006*"love"
2021-08-23 15:41:32,476 : INFO : topic #10 (0.020): 0.008*"almond" + 0.006*"oily" + 0.006*"chocolate" + 0.005*"keebler" + 0.005*"macadamia" + 0.005*"good" + 0.005*"product" + 0.005*"salt" + 0.005*"taste" + 0.005*"love"
2021-08-23 15:41:32,478 : INFO : topic #7 (0.020): 0.014*"pod" + 0.010*"win" + 0.009*"assortment" + 0.008*"betty" + 0.007*"crocker" + 0.007*"bolder" + 0.006*"coffee" + 0.006*"great" + 0.006*"jalapeno" + 0.005*"annie"
2021-08-23 15:41:32,479 : INFO : topic diff=0.378773, rho=0.186058
2021-08-23 15:41:32,775 : INFO : PROGRESS: pa

...

2021-08-23 15:43:42,006 : INFO : topic #26 (0.020): 0.089*"tea" + 0.015*"green" + 0.011*"bag" + 0.010*"flavor" + 0.009*"chai" + 0.009*"blackberry" + 0.009*"drink" + 0.008*"cup" + 0.008*"lipton" + 0.007*"vanilla"
2021-08-23 15:43:42,007 : INFO : topic #33 (0.020): 0.013*"cider" + 0.009*"kellogg" + 0.008*"ingredient" + 0.008*"sugar" + 0.007*"corn" + 0.006*"product" + 0.006*"apple" + 0.006*"organic" + 0.005*"syrup" + 0.005*"fructose"
2021-08-23 15:43:42,008 : INFO : topic #22 (0.020): 0.047*"pasta" + 0.020*"spaghetti" + 0.017*"oil" + 0.016*"barilla" + 0.013*"coconut" + 0.013*"sauce" + 0.013*"cook" + 0.010*"grain" + 0.008*"use" + 0.008*"tomato"
2021-08-23 15:43:42,009 : INFO : topic diff=0.180461, rho=0.160361
2021-08-23 15:43:42,700 : INFO : -10.896 per-word bound, 1905.7 perplexity estimate based on a held-out corpus of 1774 documents with 8259 words
2021-08-23 15:43:42,911 : INFO : PROGRESS: pass 15, dispatched chunk #0 = documents up to #2000/47774, outstanding queue size 1
2021-08-23 

2021-08-23 15:43:55,507 : INFO : PROGRESS: pass 16, dispatched chunk #3 = documents up to #8000/47774, outstanding queue size 3
2021-08-23 15:43:55,881 : INFO : PROGRESS: pass 16, dispatched chunk #4 = documents up to #10000/47774, outstanding queue size 3
2021-08-23 15:43:56,243 : INFO : PROGRESS: pass 16, dispatched chunk #5 = documents up to #12000/47774, outstanding queue size 3
2021-08-23 15:43:56,609 : INFO : PROGRESS: pass 16, dispatched chunk #6 = documents up to #14000/47774, outstanding queue size 3
2021-08-23 15:43:57,065 : INFO : PROGRESS: pass 16, dispatched chunk #7 = documents up to #16000/47774, outstanding queue size 3
2021-08-23 15:43:57,564 : INFO : PROGRESS: pass 16, dispatched chunk #8 = documents up to #18000/47774, outstanding queue size 2
2021-08-23 15:43:57,926 : INFO : PROGRESS: pass 16, dispatched chunk #9 = documents up to #20000/47774, outstanding queue size 3
2021-08-23 15:43:58,075 : INFO : merging changes from 16000 documents into a model of 47774 docume

2021-08-23 15:44:09,767 : INFO : topic #28 (0.020): 0.026*"caramel" + 0.019*"chocolate" + 0.017*"snack" + 0.017*"fudge" + 0.014*"popcorn" + 0.011*"sweet" + 0.011*"calorie" + 0.010*"drizzle" + 0.009*"satisfy" + 0.009*"portion"
2021-08-23 15:44:09,767 : INFO : topic #5 (0.020): 0.042*"cooky" + 0.030*"cookie" + 0.018*"chocolate" + 0.013*"chip" + 0.012*"oreo" + 0.012*"mint" + 0.009*"soft" + 0.007*"chewy" + 0.007*"like" + 0.007*"package"
2021-08-23 15:44:09,768 : INFO : topic #24 (0.020): 0.046*"sauce" + 0.022*"hot" + 0.019*"mango" + 0.016*"taco" + 0.013*"spicy" + 0.012*"habanero" + 0.012*"ginger" + 0.011*"heat" + 0.011*"kool" + 0.010*"jalapeno"
2021-08-23 15:44:09,769 : INFO : topic #2 (0.020): 0.039*"tuna" + 0.022*"mustard" + 0.017*"tabasco" + 0.016*"movie" + 0.013*"pepper" + 0.012*"relax" + 0.012*"wild" + 0.011*"theater" + 0.010*"bed" + 0.010*"walnut"
2021-08-23 15:44:09,770 : INFO : topic diff=0.143062, rho=0.154511
2021-08-23 15:44:10,029 : INFO : PROGRESS: pass 17, dispatched chunk #1

2021-08-23 15:44:22,820 : INFO : PROGRESS: pass 18, dispatched chunk #13 = documents up to #28000/47774, outstanding queue size 3
2021-08-23 15:44:23,231 : INFO : PROGRESS: pass 18, dispatched chunk #14 = documents up to #30000/47774, outstanding queue size 3
2021-08-23 15:44:23,633 : INFO : PROGRESS: pass 18, dispatched chunk #15 = documents up to #32000/47774, outstanding queue size 3
2021-08-23 15:44:24,060 : INFO : PROGRESS: pass 18, dispatched chunk #16 = documents up to #34000/47774, outstanding queue size 3
2021-08-23 15:44:24,542 : INFO : PROGRESS: pass 18, dispatched chunk #17 = documents up to #36000/47774, outstanding queue size 3
2021-08-23 15:44:24,712 : INFO : merging changes from 16000 documents into a model of 47774 documents
2021-08-23 15:44:24,729 : INFO : topic #1 (0.020): 0.019*"bear" + 0.018*"costco" + 0.017*"gummy" + 0.014*"club" + 0.014*"wasabi" + 0.013*"prune" + 0.011*"sam" + 0.011*"yummy" + 0.010*"haribo" + 0.009*"gummi"
2021-08-23 15:44:24,731 : INFO : topic #

2021-08-23 15:44:36,272 : INFO : topic #14 (0.020): 0.044*"cheez" + 0.040*"hummus" + 0.025*"tahini" + 0.017*"grand" + 0.016*"tummy" + 0.015*"basmati" + 0.014*"cabinet" + 0.014*"gelatin" + 0.013*"gravy" + 0.013*"beware"
2021-08-23 15:44:36,274 : INFO : topic #29 (0.020): 0.023*"creme" + 0.016*"columbian" + 0.013*"charge" + 0.012*"ghirardelli" + 0.012*"turmeric" + 0.012*"timely" + 0.012*"terrific" + 0.011*"fudge" + 0.010*"rum" + 0.009*"manner"
2021-08-23 15:44:36,275 : INFO : topic diff=0.113918, rho=0.150950
2021-08-23 15:44:36,624 : INFO : PROGRESS: pass 19, dispatched chunk #18 = documents up to #38000/47774, outstanding queue size 3
2021-08-23 15:44:37,098 : INFO : PROGRESS: pass 19, dispatched chunk #19 = documents up to #40000/47774, outstanding queue size 2
2021-08-23 15:44:37,574 : INFO : PROGRESS: pass 19, dispatched chunk #20 = documents up to #42000/47774, outstanding queue size 3
2021-08-23 15:44:38,067 : INFO : PROGRESS: pass 19, dispatched chunk #21 = documents up to #44000

# Evaluate Topic Coherence

In [14]:
top_topics = lda_model.top_topics(bow_corpus)

2021-08-23 15:44:40,466 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2021-08-23 15:44:40,477 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2021-08-23 15:44:40,489 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2021-08-23 15:44:40,502 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2021-08-23 15:44:40,521 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2021-08-23 15:44:40,535 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2021-08-23 15:44:40,553 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2021-08-23 15:44:40,569 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2021-08-23 15:44:40,583 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2021-08-23 15:44:40,597 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2021-08-23 15:44:40,612 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2021-08-23 15:44:40

In [15]:
# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / 50
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -8.3749.
[([(0.036660507, 'bar'),
   (0.025981054, 'chocolate'),
   (0.02302388, 'peanut'),
   (0.01718864, 'butter'),
   (0.012060091, 'dark'),
   (0.0082415305, 'granola'),
   (0.007835862, 'snack'),
   (0.0070581804, 'almond'),
   (0.0067771794, 'nut'),
   (0.0066288286, 'cocoa'),
   (0.0064642504, 'eat'),
   (0.006352493, 'sweet'),
   (0.006323103, 'fat'),
   (0.005940417, 'taste'),
   (0.0059357346, 'like'),
   (0.0053433296, 'good'),
   (0.005084938, 'calorie'),
   (0.0048570246, 'flavor'),
   (0.0043375315, 'candy'),
   (0.0042986795, 'love')],
  -1.5295965880537277),
 ([(0.014427535, 'sugar'),
   (0.013148586, 'stevia'),
   (0.012639881, 'splenda'),
   (0.011743792, 'sweetener'),
   (0.009079954, 'use'),
   (0.0082051335, 'tea'),
   (0.0059625553, 'product'),
   (0.0055735656, 'packet'),
   (0.005493913, 'sweeten'),
   (0.0050705844, 'taste'),
   (0.0049331514, 'sweet'),
   (0.00455018, 'artificial'),
   (0.004414875, 'aftertaste'),
   (0.0043636155, 'd

# Retrieve Topic Vectors

In [16]:
train_vecs = []
for i in tqdm(range(len(processed_reviews))):
    top_topics = lda_model.get_document_topics(bow_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(50)]
    train_vecs.append(topic_vec)

100%|███████████████████████████████████████████████████████| 47774/47774 [00:22<00:00, 2130.87it/s]


In [17]:
train_vecs[0]

[0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.9423492,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481,
 0.0011765481]

In [18]:
processed_reviews[0]

['curry',
 'paste',
 'delicious',
 'curry',
 'fry',
 'chicken',
 'vegetable',
 'add',
 'coconut',
 'milk',
 'delicious',
 'dish',
 'spicy',
 'look',
 'mild',
 'look']

In [19]:
lda_model.print_topics(50, num_words=10)

2021-08-23 15:45:04,112 : INFO : topic #0 (0.020): 0.037*"bar" + 0.026*"chocolate" + 0.023*"peanut" + 0.017*"butter" + 0.012*"dark" + 0.008*"granola" + 0.008*"snack" + 0.007*"almond" + 0.007*"nut" + 0.007*"cocoa"
2021-08-23 15:45:04,114 : INFO : topic #1 (0.020): 0.020*"costco" + 0.017*"bear" + 0.016*"gummy" + 0.014*"club" + 0.013*"wasabi" + 0.012*"prune" + 0.011*"skinny" + 0.011*"yummy" + 0.011*"sam" + 0.010*"kirkland"
2021-08-23 15:45:04,114 : INFO : topic #2 (0.020): 0.047*"tuna" + 0.024*"mustard" + 0.018*"movie" + 0.015*"tabasco" + 0.014*"relax" + 0.013*"theater" + 0.012*"wild" + 0.012*"bed" + 0.011*"collection" + 0.010*"pepper"
2021-08-23 15:45:04,116 : INFO : topic #3 (0.020): 0.012*"price" + 0.012*"amazon" + 0.010*"save" + 0.009*"store" + 0.009*"order" + 0.009*"subscribe" + 0.008*"buy" + 0.007*"product" + 0.007*"maple" + 0.007*"shipping"
2021-08-23 15:45:04,117 : INFO : topic #4 (0.020): 0.040*"soup" + 0.033*"noodle" + 0.015*"ramen" + 0.013*"miso" + 0.012*"spicy" + 0.011*"dr" + 

2021-08-23 15:45:04,177 : INFO : topic #39 (0.020): 0.041*"candy" + 0.029*"chocolate" + 0.021*"caramel" + 0.013*"werther" + 0.010*"melt" + 0.010*"kid" + 0.009*"mouth" + 0.009*"love" + 0.008*"wrap" + 0.008*"piece"
2021-08-23 15:45:04,179 : INFO : topic #40 (0.020): 0.054*"pretzel" + 0.027*"toffee" + 0.024*"cream" + 0.023*"ice" + 0.017*"fond" + 0.017*"desert" + 0.016*"flavour" + 0.016*"squirt" + 0.011*"keeper" + 0.011*"shred"
2021-08-23 15:45:04,181 : INFO : topic #41 (0.020): 0.046*"coffee" + 0.016*"starbucks" + 0.012*"cup" + 0.012*"mountain" + 0.011*"bean" + 0.010*"espresso" + 0.010*"roast" + 0.009*"cappuccino" + 0.009*"ground" + 0.009*"vanilla"
2021-08-23 15:45:04,183 : INFO : topic #42 (0.020): 0.026*"ahoy" + 0.026*"peppermint" + 0.025*"licorice" + 0.014*"mocha" + 0.010*"gross" + 0.010*"holiday" + 0.009*"addictive" + 0.008*"kit" + 0.008*"chocolate" + 0.008*"nd"
2021-08-23 15:45:04,185 : INFO : topic #43 (0.020): 0.030*"cereal" + 0.010*"snack" + 0.010*"cinnamon" + 0.010*"fiber" + 0.00

[(0,
  '0.037*"bar" + 0.026*"chocolate" + 0.023*"peanut" + 0.017*"butter" + 0.012*"dark" + 0.008*"granola" + 0.008*"snack" + 0.007*"almond" + 0.007*"nut" + 0.007*"cocoa"'),
 (1,
  '0.020*"costco" + 0.017*"bear" + 0.016*"gummy" + 0.014*"club" + 0.013*"wasabi" + 0.012*"prune" + 0.011*"skinny" + 0.011*"yummy" + 0.011*"sam" + 0.010*"kirkland"'),
 (2,
  '0.047*"tuna" + 0.024*"mustard" + 0.018*"movie" + 0.015*"tabasco" + 0.014*"relax" + 0.013*"theater" + 0.012*"wild" + 0.012*"bed" + 0.011*"collection" + 0.010*"pepper"'),
 (3,
  '0.012*"price" + 0.012*"amazon" + 0.010*"save" + 0.009*"store" + 0.009*"order" + 0.009*"subscribe" + 0.008*"buy" + 0.007*"product" + 0.007*"maple" + 0.007*"shipping"'),
 (4,
  '0.040*"soup" + 0.033*"noodle" + 0.015*"ramen" + 0.013*"miso" + 0.012*"spicy" + 0.011*"dr" + 0.011*"bowl" + 0.010*"mcdougall" + 0.008*"lunch" + 0.007*"quick"'),
 (5,
  '0.043*"cooky" + 0.032*"cookie" + 0.018*"chocolate" + 0.015*"chip" + 0.013*"mint" + 0.013*"oreo" + 0.010*"soft" + 0.008*"chewy" 

# Compute Model Coherence Score

In [23]:
# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

2021-08-23 15:46:33,015 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2021-08-23 15:46:33,030 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2021-08-23 15:46:33,044 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2021-08-23 15:46:33,063 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2021-08-23 15:46:33,084 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2021-08-23 15:46:33,106 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2021-08-23 15:46:33,128 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2021-08-23 15:46:33,147 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2021-08-23 15:46:33,166 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2021-08-23 15:46:33,184 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2021-08-23 15:46:33,203 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2021-08-23 15:46:33


Coherence Score:  -8.374925074530148


### Optimize Coherence Values

In [43]:
def compute_coherence_values(id2word, corpus, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        lda_model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=20, workers=8)
        model_list.append(lda_model)
        coherencemodel = gensim.models.CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(id2word=dictionary, corpus=bow_corpus, start=5, limit=50, step=5)

# Show graph
# limit=40; start=2; step=4;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

2021-08-23 16:36:58,103 : INFO : using symmetric alpha at 0.2
2021-08-23 16:36:58,105 : INFO : using symmetric eta at 0.2
2021-08-23 16:36:58,107 : INFO : using serial LDA version on this node
2021-08-23 16:36:58,111 : INFO : running online LDA training, 5 topics, 20 passes over the supplied corpus of 47774 documents, updating every 16000 documents, evaluating every ~47774 documents, iterating 50x with a convergence threshold of 0.001000
2021-08-23 16:36:58,112 : INFO : training LDA model using 8 processes


5


/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similar

2021-08-23 16:37:08,323 : INFO : topic #1 (0.200): 0.020*"taste" + 0.016*"good" + 0.012*"buy" + 0.012*"like" + 0.010*"flavor" + 0.010*"coffee" + 0.009*"use" + 0.008*"product" + 0.007*"chocolate" + 0.006*"eat"
2021-08-23 16:37:08,325 : INFO : topic #2 (0.200): 0.020*"like" + 0.016*"good" + 0.014*"flavor" + 0.014*"taste" + 0.014*"coffee" + 0.013*"great" + 0.013*"use" + 0.010*"try" + 0.007*"love" + 0.007*"chocolate"
2021-08-23 16:37:08,326 : INFO : topic #3 (0.200): 0.019*"flavor" + 0.016*"taste" + 0.013*"like" + 0.012*"good" + 0.011*"use" + 0.009*"coffee" + 0.009*"great" + 0.008*"try" + 0.007*"buy" + 0.007*"product"
2021-08-23 16:37:08,327 : INFO : topic #4 (0.200): 0.019*"taste" + 0.016*"like" + 0.015*"product" + 0.014*"good" + 0.014*"tea" + 0.011*"use" + 0.011*"flavor" + 0.009*"love" + 0.008*"try" + 0.007*"eat"
2021-08-23 16:37:08,328 : INFO : topic diff=0.352404, rho=0.333333
2021-08-23 16:37:10,887 : INFO : -7.238 per-word bound, 151.0 perplexity estimate based on a held-out corpus o

2021-08-23 16:37:19,351 : INFO : topic #0 (0.200): 0.035*"tea" + 0.017*"like" + 0.017*"good" + 0.015*"taste" + 0.014*"flavor" + 0.014*"cup" + 0.013*"coffee" + 0.011*"use" + 0.009*"add" + 0.008*"try"
2021-08-23 16:37:19,352 : INFO : topic #1 (0.200): 0.020*"taste" + 0.016*"good" + 0.013*"like" + 0.012*"chocolate" + 0.011*"buy" + 0.010*"flavor" + 0.008*"use" + 0.008*"product" + 0.007*"bar" + 0.006*"eat"
2021-08-23 16:37:19,353 : INFO : topic #2 (0.200): 0.023*"like" + 0.017*"good" + 0.016*"taste" + 0.015*"flavor" + 0.015*"coffee" + 0.013*"great" + 0.011*"use" + 0.010*"try" + 0.010*"chocolate" + 0.008*"sugar"
2021-08-23 16:37:19,354 : INFO : topic #3 (0.200): 0.020*"flavor" + 0.016*"taste" + 0.014*"like" + 0.013*"good" + 0.011*"use" + 0.009*"great" + 0.009*"sauce" + 0.008*"try" + 0.008*"buy" + 0.007*"product"
2021-08-23 16:37:19,354 : INFO : topic #4 (0.200): 0.021*"taste" + 0.017*"like" + 0.016*"product" + 0.014*"good" + 0.011*"flavor" + 0.009*"use" + 0.009*"drink" + 0.008*"try" + 0.008*

...

2021-08-23 17:00:55,246 : INFO : topic #25 (0.022): 0.072*"product" + 0.039*"good" + 0.037*"taste" + 0.027*"brand" + 0.019*"buy" + 0.017*"like" + 0.017*"star" + 0.014*"try" + 0.013*"think" + 0.012*"quality"
2021-08-23 17:00:55,261 : INFO : topic #2 (0.022): 0.036*"pop" + 0.031*"licorice" + 0.021*"bear" + 0.017*"ring" + 0.016*"taste" + 0.016*"bag" + 0.016*"gummy" + 0.012*"like" + 0.012*"peppermint" + 0.011*"good"
2021-08-23 17:00:55,268 : INFO : topic #6 (0.022): 0.090*"bar" + 0.062*"peanut" + 0.043*"butter" + 0.034*"granola" + 0.022*"like" + 0.021*"taste" + 0.020*"nut" + 0.017*"good" + 0.016*"eat" + 0.015*"snack"
2021-08-23 17:00:55,285 : INFO : topic diff=0.259516, rho=0.158338
2021-08-23 17:00:57,068 : INFO : -6.910 per-word bound, 120.3 perplexity estimate based on a held-out corpus of 1774 documents with 78463 words
2021-08-23 17:00:57,106 : INFO : merging changes from 15774 documents into a model of 47774 documents
2021-08-23 17:00:57,120 : INFO : topic #35 (0.022): 0.093*"water" 

2021-08-23 17:01:05,229 : INFO : topic #37 (0.022): 0.116*"tea" + 0.059*"green" + 0.031*"drink" + 0.029*"aloe" + 0.017*"leaf" + 0.017*"taste" + 0.014*"benefit" + 0.012*"like" + 0.012*"good" + 0.012*"organic"
2021-08-23 17:01:05,230 : INFO : topic #33 (0.022): 0.142*"coffee" + 0.026*"like" + 0.025*"flavor" + 0.022*"taste" + 0.021*"roast" + 0.018*"good" + 0.017*"blend" + 0.015*"strong" + 0.014*"cup" + 0.013*"starbucks"
2021-08-23 17:01:05,230 : INFO : topic #6 (0.022): 0.092*"bar" + 0.062*"peanut" + 0.046*"butter" + 0.038*"granola" + 0.022*"like" + 0.022*"taste" + 0.018*"nut" + 0.017*"good" + 0.016*"eat" + 0.015*"snack"
2021-08-23 17:01:05,231 : INFO : topic #21 (0.022): 0.186*"tea" + 0.030*"bag" + 0.022*"flavor" + 0.015*"taste" + 0.014*"like" + 0.013*"green" + 0.012*"cup" + 0.011*"vanilla" + 0.011*"lipton" + 0.011*"good"
2021-08-23 17:01:05,233 : INFO : topic diff=0.236253, rho=0.156389
2021-08-23 17:01:06,460 : INFO : -6.889 per-word bound, 118.5 perplexity estimate based on a held-out

2021-08-23 17:01:14,411 : INFO : PROGRESS: pass 18, dispatched chunk #0 = documents up to #2000/47774, outstanding queue size 1
2021-08-23 17:01:14,422 : INFO : PROGRESS: pass 18, dispatched chunk #1 = documents up to #4000/47774, outstanding queue size 2
2021-08-23 17:01:14,424 : INFO : PROGRESS: pass 18, dispatched chunk #2 = documents up to #6000/47774, outstanding queue size 3
2021-08-23 17:01:14,425 : INFO : PROGRESS: pass 18, dispatched chunk #3 = documents up to #8000/47774, outstanding queue size 4
2021-08-23 17:01:14,434 : INFO : PROGRESS: pass 18, dispatched chunk #4 = documents up to #10000/47774, outstanding queue size 5
2021-08-23 17:01:14,437 : INFO : PROGRESS: pass 18, dispatched chunk #5 = documents up to #12000/47774, outstanding queue size 6
2021-08-23 17:01:14,439 : INFO : PROGRESS: pass 18, dispatched chunk #6 = documents up to #14000/47774, outstanding queue size 7
2021-08-23 17:01:14,449 : INFO : PROGRESS: pass 18, dispatched chunk #7 = documents up to #16000/4777

2021-08-23 17:01:22,567 : INFO : PROGRESS: pass 19, dispatched chunk #8 = documents up to #18000/47774, outstanding queue size 9
2021-08-23 17:01:22,568 : INFO : PROGRESS: pass 19, dispatched chunk #9 = documents up to #20000/47774, outstanding queue size 10
2021-08-23 17:01:22,578 : INFO : PROGRESS: pass 19, dispatched chunk #10 = documents up to #22000/47774, outstanding queue size 11
2021-08-23 17:01:22,582 : INFO : PROGRESS: pass 19, dispatched chunk #11 = documents up to #24000/47774, outstanding queue size 12
2021-08-23 17:01:22,583 : INFO : PROGRESS: pass 19, dispatched chunk #12 = documents up to #26000/47774, outstanding queue size 13
2021-08-23 17:01:22,584 : INFO : PROGRESS: pass 19, dispatched chunk #13 = documents up to #28000/47774, outstanding queue size 14
2021-08-23 17:01:22,600 : INFO : PROGRESS: pass 19, dispatched chunk #14 = documents up to #30000/47774, outstanding queue size 15
2021-08-23 17:01:22,601 : INFO : PROGRESS: pass 19, dispatched chunk #15 = documents u

2021-08-23 17:01:31,075 : INFO : CorpusAccumulator accumulated stats from 20000 documents
2021-08-23 17:01:31,093 : INFO : CorpusAccumulator accumulated stats from 21000 documents
2021-08-23 17:01:31,112 : INFO : CorpusAccumulator accumulated stats from 22000 documents
2021-08-23 17:01:31,132 : INFO : CorpusAccumulator accumulated stats from 23000 documents
2021-08-23 17:01:31,150 : INFO : CorpusAccumulator accumulated stats from 24000 documents
2021-08-23 17:01:31,170 : INFO : CorpusAccumulator accumulated stats from 25000 documents
2021-08-23 17:01:31,194 : INFO : CorpusAccumulator accumulated stats from 26000 documents
2021-08-23 17:01:31,213 : INFO : CorpusAccumulator accumulated stats from 27000 documents
2021-08-23 17:01:31,230 : INFO : CorpusAccumulator accumulated stats from 28000 documents
2021-08-23 17:01:31,247 : INFO : CorpusAccumulator accumulated stats from 29000 documents
2021-08-23 17:01:31,272 : INFO : CorpusAccumulator accumulated stats from 30000 documents
2021-08-23

In [44]:
print(coherence_values)

[-1.976139847069205, -1.9930238769459248, -2.1492684446983885, -2.139310359835681, -2.177826543155738, -2.3729310418039464, -2.42461473417688, -2.373906710624463, -2.418559580987307]


In [53]:
optimal_model = model_list[3]

In [54]:
optimal_model.show_topics(num_topics=20)

[(0,
  '0.047*"cereal" + 0.027*"like" + 0.022*"taste" + 0.018*"good" + 0.017*"flavor" + 0.017*"eat" + 0.016*"sweet" + 0.016*"cinnamon" + 0.016*"sugar" + 0.013*"oatmeal"'),
 (1,
  '0.112*"chocolate" + 0.028*"dark" + 0.020*"bar" + 0.020*"caramel" + 0.018*"cocoa" + 0.018*"like" + 0.017*"almond" + 0.017*"taste" + 0.015*"good" + 0.015*"candy"'),
 (2,
  '0.036*"product" + 0.018*"taste" + 0.014*"review" + 0.012*"like" + 0.010*"good" + 0.009*"package" + 0.009*"star" + 0.009*"open" + 0.009*"try" + 0.009*"know"'),
 (3,
  '0.085*"coffee" + 0.029*"cup" + 0.028*"like" + 0.027*"taste" + 0.026*"flavor" + 0.024*"drink" + 0.017*"good" + 0.013*"try" + 0.010*"strong" + 0.009*"energy"'),
 (4,
  '0.049*"bar" + 0.027*"seed" + 0.021*"fat" + 0.016*"calorie" + 0.016*"protein" + 0.015*"nut" + 0.014*"high" + 0.014*"fiber" + 0.013*"eat" + 0.013*"food"'),
 (5,
  '0.119*"tea" + 0.031*"flavor" + 0.021*"taste" + 0.021*"like" + 0.018*"green" + 0.017*"drink" + 0.015*"bag" + 0.012*"good" + 0.011*"try" + 0.010*"cup"'),
 

# Visualize with pyLDAvis

In [71]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
viz = pyLDAvis.gensim_models.prepare(optimal_model, bow_corpus, dictionary)

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/li

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.046300 -0.064228       1        1  10.193488
2     -0.006599  0.001508       2        1   8.103247
8     -0.137077 -0.052087       3        1   7.209367
5      0.053373 -0.055079       4        1   6.869419
11     0.185350  0.165591       5        1   6.254319
6      0.132937 -0.061623       6        1   5.799964
0      0.115601 -0.040445       7        1   5.773801
10    -0.044204 -0.101695       8        1   5.072331
1      0.098425 -0.120175       9        1   4.984245
9     -0.031457 -0.041782      10        1   4.890623
19     0.089321 -0.156113      11        1   4.679159
17     0.020144 -0.005938      12        1   4.534896
13    -0.064209  0.016906      13        1   4.208183
4      0.126402  0.154632      14        1   4.039228
15    -0.073559  0.099192      15        1   3.619300
14    -0.059081  0.037929      16        1   3.337801
18    -0.115879 -0.026514      17        1   3.180962
12    -0.182158  0.062645      18        1   2.539258
7     -0.006399  0.199030      19        1   2.365590
16    -0.147233 -0.011755      20        1   2.344819, topic_info=          Term          Freq         Total Category  logprob  loglift
320        tea  15219.000000  15219.000000  Default  30.0000  30.0000
397     coffee  16118.000000  16118.000000  Default  29.0000  29.0000
434  chocolate  12806.000000  12806.000000  Default  28.0000  28.0000
30         use  18885.000000  18885.000000  Default  27.0000  27.0000
349      drink  11673.000000  11673.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
194      order    273.866169   5177.735347  Topic20  -5.0722   0.8135
43       taste    383.971773  33165.338152  Topic20  -4.7343  -0.7057
37        long    243.720748   3444.536627  Topic20  -5.1889   1.1044
82        need    242.797920   5301.003615  Topic20  -5.1927   0.6695
170    product    271.961716  15956.177547  Topic20  -5.0792  -0.3190

[1434 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
5537     14  0.144365   aback
5537     17  0.818070   aback
834       1  0.095513    acid
834       4  0.018851    acid
834       5  0.601985    acid
...     ...       ...     ...
4297      2  0.017976    zico
4297     17  0.970720    zico
5052      8  0.168191  zipper
5052     18  0.798906  zipper
5232     13  0.970933    ziti

[6735 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 9, 6, 12, 7, 1, 11, 2, 10, 20, 18, 14, 5, 16, 15, 19, 13, 8, 17])

In [ ]:
viz